## 트리의 앙상블

### 정형 데이터와 비정형 데이터

졍형 데이터(structured data)
- 어떤 구조로 되어 있는 데이터

- CSV나 데이터베이스, 엑셀에 저장하기 쉬움
- 프로그래머가 다루는 대부분의 데이터가 정형 데이터임

비정형 데이터(unstructure data)
- 데이터베이스나 엑셀로 표현하기 어려운 데이터

- 텍스트 데이터, 사진, 음악 등

앙상블 학습(ensemble learning)
- 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

- 결정 트리 기반으로 만들어짐

- 비정형 데이터 -> 신경망 알고리즘 사용

### 랜덤 포레스트

랜덤 포레스트(random forest)
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다

- 각 결정 트리의 예측을 사용해 최종 예측을 만든다

- 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하데 만든다

- 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦
- - 한 샘플이 중복되어 추출될 수 있음

부트스트랩 샘플(bootstrap sample)
- 데이터 세트에서 중복을 허용하여 데이터를 샘플링하여 분류한 데이터


- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음

- 분류 모델 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼 특성을 선택

- 회귀 모델 RandomForestRegressor는 전체 특성을 사용

- 기본적으로 100개의 결정 트리를 이러한 방식으로 훈련

- 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음
- 회귀일 때는 단순히 각 트리의 예측을 평균

- 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합괴는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음

In [1]:
# RandomForestClassifier 클래스를 와인 분류 문제에 적용
# 훈련 세트와 테스트 세트로 나누기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [2]:
# 병렬로 교차 검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8903229806766861


- 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공

- 특성 중요도를 계산 -> 각 결정 트리의 특성 중요도를 취합한 것

In [3]:
# 랜덤 포레스트 모델을 훈련 세트에 훈련
rf.fit(train_input, train_target)
# 특성 중요도 출력
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


- RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있는 기능이 있음

- 후련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리를 평가
- 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있음 -> OOB(out of bag)
- 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리 평가 가능

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


### 엑스트라 트리

엑스트라 트리(Extra Trees)
- 랜덤 포레스트와 매우 비슷하게 동작
- 기본적으로 100개의 결정 트리를 훈련
- 결정 트리가 제공하는 대부분의 매개변수를 지원

- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용

랜덤 포레스트와 엑스트라 트리의 차이점
- 부트스트랩 샘플을 사용하지 않는다

- 각 결정 트리를 만들 때 전체 훈련 세트를 사용
- 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할

In [5]:
# 엑스트라 트리의 교차 검증 점수 확인
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


- 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함

- 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점

- 결정 트리는 최적의 분할을 찾는 데 시간을 많이 소모함 -> 특성의 개수가 많을 때 더 그럼

- 무작위로 나눈다면 훨씬 빨리 트리 구성 가능

In [6]:
# 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


- 엑스트라 트리의 회귀 버전 -> ExtraTreesRegressor 클래스

### 그레이디언트 부스팅

그레이디언트 부스팅(gradient boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법

- GradientBoostingClassifier은 기본적으로 깊이가 3인 결정 트리를 100개 사용
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대 적합에 강하고 일반적으로 높은 일반화 성능 기대 가능

- 경사 하강법을 사용하여 트리를 앙상블에 추가

- 분류에서는 로지스틱 손실 함수를 사용
- 회귀에서는 평균 제곱 오차 함수를 사용

- 결정 트리를 계속 추가하면서 가장 낮을 곳을 찾아 이동

- 학습률 매개변수로 속도를 조절

In [7]:
# 와인 데이터셋의 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
# 그레이디언트 부스팅 분류 모델 생성
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
# 와인 데이터셋 교차 검증 점수 확인
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


- 결정 트리의 개수를 늘려도 과대적합에 매우 강함

- 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

In [11]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
# 특성 중요도 확인
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


subsample 
- 트리 훈련에 사용할 훈련 세트의 비율을 정하는 매개변수
- 기본값 -> 1.0
- 전체 훈련 세트를 사용

- subsample이 1보다 작으면 훈련 세트의 일부를 사용

- 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음

- 하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림
- GradientBoostingRegressor에는 n_job 매개변수가 없음

GradientBoostingRegressor
- 그레이디언트 부스팅의 회귀 버전

- 그레이디언트 부스팅의 속도와 성능을 더욱 개선한 것

### 히스토그램 기반 그레이디언트 부스팅

히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
- 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 않은 알고리즘


1. 입력 특성을 256개의 구간으로 나눔
- 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
2. 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용
- 입력에 누락된 특성이 있더라도 이를 따로 전처리 할 필요 없음

HistGradientBoostingClassifier
- 히스토그램 기반 그레이디언트 부스팅 클래스

- 기본 매개변수에서 안정적인 성능을 얻을 수 있음
- 트리의 개수를 지정하는데 n_estimators 대신에 부스팅 반복 횟수를 지정하는 max_iter를 사용
- 성능을 높이려면 max_iter 매개변수 테스트

- 사이킷런 1.0에서 히스토그램 기반 부스팅이 experimental 모듈 아래에서 벗어났으므로 eanble_hist_gradient_boosting 모듈을 임포트할 필요가 없음

In [14]:
# 와인 데이터셋에 HistGradientBoostingClassifier 적용
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/Users/youju/anaconda3/lib/python3.11/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


- 과대적합을 잘 억제하면서 그레이티언트 부스팅보다 조금 더 높은 성능을 제공

- permutation_importance() : 히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산할 때 사용
- 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산
- 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델이 모두 사용 가능

In [16]:
# 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1) # 10번 랜덤하게 섞음
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
# 테스트 세트 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [18]:
# HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

- 앙상블 모델이 단일 결정 트리보다 좋은 결과를 얻을 수 있음

HistGradientBoostingRegressor
- 히스토그램 기반 그레이디언트 부스팅 회귀버전

XGBoost
- 부스팅 알고리즘을 구현한 라이브러리 중 하나

- 코랩에서 사용 가능, 사이킷런의 cross_validate() 함수와 함께 사용 가능
- 다양한 부스팅 알고리즘 지원
- tree_method 매개변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅 사용 가능

In [22]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/52/39/ec5c53228b091387e934d3d419e8e3a5ce98c1650d458987d6e254a15304/xgboost-3.1.3-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00a 0:00:01


In [23]:
# 와인 데이터의 교차 검증 점수 확인
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


LightGBM
- 마이크로소프트에서 만든 히스토그램 기반 그레이디언트 부스팅 라이브러리

In [24]:
! pip install lightGBM

  Obtaining dependency information for lightGBM from https://files.pythonhosted.org/packages/21/1b/550ee378512b78847930f5d74228ca1fdba2a7fbdeaac9aeccc085b0e257/lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.6 MB/s eta 0:00:00a 0:00:01


In [25]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3152, number of negative: 1006
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 371
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000877 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.758057 -> initscore=1.142055
[LightGBM] [Info] Start training from score 1.142055
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757816 -> initscore=1.140744
[LightGBM] [Info] Start training from score 1.140744
[LightGBM] [Info] Number o